In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==2.3 transformers==4.39.0 appdirs jsonpickle filelock h5py spacy nltk pytest radgraph tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import shutil

src_path = "/content/drive/MyDrive/Dual view Slava/radgraph-xl.tar.gz"
cache_dir = "/root/.cache/radgraph/0.1.2"
dst_path = f"{cache_dir}/radgraph-xl.tar.gz"

os.makedirs(cache_dir, exist_ok=True)

shutil.copy(src_path, dst_path)

print("Model copied to RadGraph cache location.")


In [ ]:
import json
from tqdm import tqdm
from radgraph import RadGraph

def initialize_radgraph():
    try:
        return RadGraph(model_type="radgraph-xl", device="cpu")
    except Exception as e:
        print(f"Failed to initialize RadGraph: {e}")
        return None



def clean_text_with_context(text, radgraph_instance):
    if not text.strip():
        return ""

    try:
        annotations = radgraph_instance([text])
        entities = annotations["0"].get("entities", {})
        important_tokens = set()
        for ent in entities.values():

            if ent["label"] not in ("O",):
                important_tokens.add(ent["tokens"].strip())


        sentences = [s.strip() for s in text.replace('\n', ' ').split('.') if s.strip()]


        kept_sentences = []
        for sentence in sentences:
            if "_" in sentence:
                continue
            for token in important_tokens:
                if token in sentence:
                    kept_sentences.append(sentence.strip())
                    break

        return '. '.join(kept_sentences) + '.' if kept_sentences else ""

    except Exception as e:
        print(f"Error processing text: {e}")
        return ""


def main():
    rg = initialize_radgraph()
    if not rg:
        return

    try:
        with open("cleaned_reports.json") as f:
            data = json.load(f)
    except Exception as e:
        print(f"Failed to load input file: {e}")
        return


    for study_id, entry in tqdm(data.items()):

        try:
            entry["findings"] = clean_text_with_context(entry.get("findings", ""), rg)
            entry["impression"] = clean_text_with_context(entry.get("impression", ""), rg)

        except Exception as e:
            print(f"Error processing entry {study_id}: {e}")
            continue

    try:
        with open("cleaned_reports_clinical.json", "w") as f:
            json.dump(data, f, indent=2)

    except Exception as e:
        print(f"Failed to save results: {e}")

if __name__ == "__main__":
    main()

Using device: cpu


 26%|██▌       | 8562/32734 [1:03:21<2:37:12,  2.56it/s]/usr/local/lib/python3.11/dist-packages/radgraph/dygie/data/dataset_readers/dygie.py:189: UserWarning: Document 0 has a sentence with a single token or no tokens. This may break the modeling code.
  warnings.warn(msg)
100%|██████████| 32734/32734 [4:25:56<00:00,  2.05it/s]


In [ ]:
import json

def has_short_sentence(text):
    if not text.strip():
        return True
    return False

def main():
    try:
        with open("cleaned_reports_clinical.json") as f:
            data = json.load(f)
    except Exception as e:
        print(f"Failed to load file: {e}")
        return


    cleaned_data = {}
    empty_findings_count = 0
    empty_impression_count = 0
    both_empty_count = 0

    for study_id, entry in data.items():
        findings = entry.get("findings", "")
        impression = entry.get("impression", "")

        findings_empty = not findings.strip()
        impression_empty = not impression.strip()

        if findings_empty and impression_empty:
            both_empty_count += 1
        elif findings_empty:
            empty_findings_count += 1
        elif impression_empty:
            empty_impression_count += 1


        if not findings_empty and not impression_empty:
            cleaned_data[study_id] = entry


    try:
        with open("cleaned_radgraph_output.json", "w") as f:
            json.dump(cleaned_data, f, indent=2)


        print(f"Original studies count: {len(data)}")
        print(f"Cleaned studies count: {len(cleaned_data)}")
        print(f"Removed studies with empty findings: {empty_findings_count}")
        print(f"Removed studies with empty impressions: {empty_impression_count}")
        print(f"Removed studies with both empty: {both_empty_count}")
        print(f"Total removed: {empty_findings_count + empty_impression_count + both_empty_count}")

    except Exception as e:
        print(f"Failed to save cleaned file: {e}")

if __name__ == "__main__":
    main()

Original studies count: 32734
Cleaned studies count: 31830
Removed studies with empty findings: 179
Removed studies with empty impressions: 725
Removed studies with both empty: 0
Total removed: 904
